# 2.3 NumPy数组的计算：通用函数
NumPy数组的计算有时非常快，有时也非常慢。使NumPy变快的关键是利用 **向量化** 操作，通常在NumPy的 **通用函数(ufunc)** 中实现。本节将介绍Numpy通用函数的重要性——它可以提高数组元素的重复计算的效率；然后，将会介绍很多NumPy包中常用且有用的数学通用函数。

## 缓慢的循环
Python的相对缓慢通常出现在很多小操作需要不断重复地时候，比如对数组的每个元素做循环操作时。假设有一个数组，我们想计算每个数组的倒数，一种直接的解决方法是：

In [3]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

这种实现方式可能对于有C语言或Java背景的人来说非常自然，但是如果测试一个很大量的输入数据运行上述代码的时间，这一操作将非常耗时，并且出乎意料的慢！我们将用IPython的`%timeit`魔法函数来测量：

In [5]:
big_array = np.random.randint(1,100,size=1000000)
%timeit compute_reciprocals(big_array)

1.69 s ± 9.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


事实上，这里的处理瓶颈并不是运算本身，而是CPython在每次循环时必须做数据类型的检查和函数的调度。每次进行倒数运算时，Python首先检查对象的类型，并且动态查找可以使用该数据类型的正确函数。如果我们在编译代码时进行这样的操作，那么就能在代码执行之前知晓类型的声明，结果的计算也会更加有效率。

## 通用函数介绍
NumPy为很多类型的操作提供了非常方便的、静态类型的、可编译程序的接口，也被称作向量操作。这种向量方法被用于将循环推送至NumPy之下的编译层，这样会取得更快的执行效率。

比较以下两个结果：

In [8]:
print(compute_reciprocals(values))
print(1.0/values)

[0.16666667 1.         0.25       0.25       0.125     ]
[0.16666667 1.         0.25       0.25       0.125     ]


如果计算一个较大数组的运行时间，可以看到它的完成时间比Python循环花费的时间更短：

In [9]:
%timeit (1.0/values)

955 ns ± 22.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


NumPy中的向量操作是通过**通用函数**实现的。通用函数的主要目的是对NumPy数组中的值执行更快的操作。它非常灵活，前面我们看过了标量和数组的运算，但是也可以对两个数组进行运算：

In [10]:
np.arange(5) / np.arange(1,6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

通用函数并不仅限于一维数组的运算，它们也可以进行多维数组的运算：

In [14]:
x = np.arange(9).reshape((3,3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]], dtype=int32)

通过通用函数用向量的方式进行计算几乎总比用Python循环实现的计算更有效，尤其是当数组很大时。只要你看到Python脚本中有这样的循环，就应该考虑能都用向量方式替换这个循环。

## 探索NumPy的通用函数
通用函数有两种存在形式：**一元通用函数（unary ufunc）** 对单个输入操作，**二元通用函数（binary ufunc）** 对两个输入操作。

### 数组的运算
NumPy通用函数的使用方式非常自然，因为它用到了Python原生的算术运算符，标准的加、减、乘、除都可以使用：

x = np.arange(4)

print("x      =",x)
print("x + 5  =",x + 5)
print("x - 5  =",x - 5)
print("x * 2  =",x * 2)
print("x / 2  =",x / 2)
print("x // 2 =",x // 2) #向下整除运算

还有求负数、** 表示的指数运算和%表示的模运算符的一元通用函数：

In [17]:
print("-x      =",-x)
print("x ** 2  =",x ** 2)
print("x % 2   =",x % 2)

-x      = [ 0 -1 -2 -3]
x ** 2  = [0 1 4 9]
x % 2   = [0 1 0 1]


可以任意将这些算术运算符组合使用。当然，得考虑这些运算符的优先级：

In [18]:
-(0.5 * x + 1) ** 2

array([-1.  , -2.25, -4.  , -6.25])

所有的这些算术运算符都是NumPy内置函数的简单封装器，例如+运算符就是一个add函数的封装器：

In [19]:
np.add(x,2)

array([2, 3, 4, 5])

下面列出了所有NumPy实现的算术运算符：

| 运算符 | 对应的通用函数 | 描述 |
|  :-----|  :-----| :-----|
| + | np.add | 加法运算（即 1 + 1 = 2）|
| - | np.substract | 减法运算（即 3 - 2 = 1）|
| - | np.negative | 负数运算（即 -2）|
| * | np.multiply | 乘法运算（即 2 * 3 = 6）|
| / | np.divide | 除法运算（即 3 / 2 = 1.5）|
| // | np.floor_divide | 向下整除运算（即 3 // 2 = 1）|
| ** | np.power | 指数运算（即 2 ** 3 = 8）|
| % | np.mod | 模/余数（即 9 % 4 = 1）|

另外，NumPy中还有布尔/位运算符，这些运算符将在2.6节中进一步介绍。

### 绝对值

In [21]:
x = np.array([-2,-1,0,1,2])
abs(x)

array([2, 1, 0, 1, 2])

对应的NumPy通用函数是`np.absolute`，该函数也可以用别名`np.abs`来访问：

In [23]:
np.absolute(x)

array([2, 1, 0, 1, 2])

In [24]:
np.abs(x)

array([2, 1, 0, 1, 2])

这个通用函数也可以处理复数。当处理复数时，绝对值返回的是该复数的模（magnitude）：

In [25]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

array([5., 5., 2., 1.])

### 三角函数
NumPy提供了大量好用的通用函数，其中对于数据科学家最有用的就是三角函数。首先定义一个角度数组：

In [26]:
theta = np.linspace(0,np.pi,3)

现在可以对这些值进行一些三角函数计算：

In [27]:
print("theta      =",theta)
print("sin(theta) =",np.sin(theta))
print("cos(theta) =",np.cos(theta))
print("tan(theta) =",np.tan(theta))

theta      = [0.         1.57079633 3.14159265]
sin(theta) = [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) = [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) = [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]


这些值是在机器精度内计算的，所有有些应该是0的值并没有精确到0。逆三角函数同样可以使用：

In [29]:
x = [-1,0,1]

print("x         =",x)
print("arcsin(x) =",np.arcsin(x))
print("arccos(x) =",np.arccos(x))
print("arctan(x) =",np.arctan(x))

x         = [-1, 0, 1]
arcsin(x) = [-1.57079633  0.          1.57079633]
arccos(x) = [3.14159265 1.57079633 0.        ]
arctan(x) = [-0.78539816  0.          0.78539816]


### 指数和对数

In [30]:
x = [1,2,3]

print("x   =",x)
print("e^x =",np.exp(x))
print("2^x =",np.exp2(x))
print("2^x =",np.power(3,x))

x   = [1, 2, 3]
e^x = [ 2.71828183  7.3890561  20.08553692]
2^x = [2. 4. 8.]
2^x = [ 3  9 27]


指数的运算的逆运算，即对数运算也是可用的。最基本的`np.log`给出的是以自然常熟（e）为底的对数。如果你希望计算以2为底数或者以10为底数的对数，可以按照如下示例处理：

In [31]:
x = [1,2,4,10]

print("x        =",x)
print("ln(x)    =",np.log(x))
print("log2(x)  =",np.log2(x))
print("log10(x) =",np.log10(x))

x        = [1, 2, 4, 10]
ln(x)    = [0.         0.69314718 1.38629436 2.30258509]
log2(x)  = [0.         1.         2.         3.32192809]
log10(x) = [0.         0.30103    0.60205999 1.        ]


还有一些特殊版本，对于非常小的输入值可以保持较好的精度：

In [32]:
x = [0, 0.001, 0.01, 0.1]

print("exp(x) - 1)  =",np.expm1(x))
print("log(1+ x) =",np.log1p(x))

exp(x) - 1)  = [0.         0.0010005  0.01005017 0.10517092]
log(1+ x) = [0.         0.0009995  0.00995033 0.09531018]


当x的值很小时，以上函数给出的值比`np.log`和`np.exp`的计算更准确。

### 专用的通用函数
NumPy还提供了很多通用函数，包括双曲三角函数、比特位运算、比较运算符、弧度转化为角度的运算的。可以浏览NumPy的文档了解。  
  
这里还有一个更加专用，也更加晦涩的通用函数优异来源是子模块`scipy.special`。如果你希望对你的数据进行一个更晦涩的数学计算，`scipy.special`可能包含了你需要的计算函数。这些函数能列一个常常的列表，下面的代码片段展示了一些可能在统计学中用到的函数：

In [34]:
from scipy import special

In [35]:
# Gamm函数（广义阶乘，generalized factorials）和相关函数
x = [1,5,10]

print('gamma(x)      =',special.gamma(x))
print('ln|gamma(x)|  =',special.gammaln(x))
print('beta(x,2)     =',special.beta(x,2))

gamma(x)      = [1.0000e+00 2.4000e+01 3.6288e+05]
ln|gamma(x)|  = [ 0.          3.17805383 12.80182748]
beta(x,2)     = [0.5        0.03333333 0.00909091]


In [37]:
#误差函数（高斯积分）
#它的实现和它的逆实现
x = np.array([0,0.3,0.7,1.0])

print('erf(x)    =',special.erf(x))
print('erfc(x)   =',special.erfc(x))
print('erfinv(x) =',special.erfinv(x))

erf(x)    = [0.         0.32862676 0.67780119 0.84270079]
erfc(x)   = [1.         0.67137324 0.32219881 0.15729921]
erfinv(x) = [0.         0.27246271 0.73286908        inf]


NumPy和`scipy.special`中提供了大量的通用函数，这些包的文档在网上就可以查到，搜索"gamma function python"即可。

## 高级的通用函数特性

### 指定输出
在进行大量运算时，有时候指定一个用于存放结果的数组是非常有用的。不同于创建临时数组，你可以用这个特性将计算结果直接写到你期望的存储位置。所有的通用函数都可以通过out参数来指定计算结果的存放位置：

In [39]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x,10,out=y)
print(y)

[ 0. 10. 20. 30. 40.]


这个特性也可以被用作数组视图，例如可以将计算结果写入指定数组的每隔一个元素的位置：

In [41]:
y = np.zeros(10)
np.power(2,x,out=y[::2])
print(y)

[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


如果这里写的是y[::2] = 2 ** x,那么结果将是创建一个临时数组，该数组存放的是 2 ** x的结果，并且接下来会将这些值复制到y数组中。对于上述例子中比较小的计算量来说，这两种方式的差别不大。但是对于较大的数组，通过慎重使用out参数将能够有效节约内存。

### 聚合
二元通用函数有些非常有趣的聚合功能，这些聚合功能可以直接在对象上计算。  
  
例如，如果我们希望用一个特定的运算`reduce`一个数组，那么可以用任何通用函数的`reduce`方法。一个`reduce`方法会对给定的元素和操作重复执行，直到得到单个的结果。

例如，对`add`通用函数调用`reduce`方法会返回数组中所有元素的和：

In [42]:
x = np.arange(1,6)
np.add.reduce(x)

15

同样，对`multiply`通用函数调用`reduce`方法会返回数组中所有元素的乘积：

In [43]:
np.multiply.reduce(x)

120

如果需要存储每次计算的中间结果，可以使用`accumulate`:

In [44]:
np.add.accumulate(x)

array([ 1,  3,  6, 10, 15], dtype=int32)

In [46]:
np.multiply.accumulate(x)

array([  1,   2,   6,  24, 120], dtype=int32)

请注意，一些特殊情况中，NumPy提供了专用函数（np.sum、np.prod、np.cumsum、np.cumprod），它们也可以实现以上`reduce`的功能，这些函数将在2.4节中具体介绍。

### 外积 
最后，任何通用函数都可以用`outer`方法获得两个不同输入数组所有元素对的函数运算结果。这意味着你可以用一行代码实现一个乘法表：

In [47]:
x = np.arange(1,6)
np.multiply.outer(x,x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])

2.7节将介绍非常有用的`ufunc.at`和`ufunc.reduceat`方法。

通用函数另一个非常有用的特性是它能操作不同大小和形状的数组，一组这样的操作被称为广播（broadcasting）。这一主题将在2.5节被介绍。

### 通用函数：更多的信息
有关通用函数的更多信息（包括可用的通用函数的完成列表），可以在NumPy(http://www.numpy.org)和SciPy（http://www.scipy.org )文档的网站找到。 前面介绍过，可以直接在IPython中通过导入相应的包，然后利用IPython的Tab键补全和帮助(?）功能获取信息。